In [42]:
%pip install pandas numpy python-chess 
%pip install tqdm ipywidgets widgetsnbextension

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [48]:
import chess
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

pgn_filename = "dataset/lichess_db_standard_rated_2025-11.pgn"
GAMES_TO_PROCESS = 500

In [49]:
piece_death_heatmaps = {
    chess.PAWN: np.zeros((8, 8)),
    chess.KNIGHT: np.zeros((8, 8)),
    chess.BISHOP: np.zeros((8, 8)),
    chess.ROOK: np.zeros((8, 8)),
    chess.QUEEN: np.zeros((8, 8)),
    chess.KING: np.zeros((8, 8)),
}


def count_death(game: chess.pgn.Game):
    board = game.board()

    for move in game.mainline_moves():
        if board.is_capture(move):
            dest = move.to_square
            # 注意：python-chess 的 rank 0 是第一行(白方底线)，但在矩阵中通常 row 0 是最上面
            # 所以 row = 7 - rank 是对的，用于匹配视觉习惯
            row, col = 7 - chess.square_rank(dest), chess.square_file(dest)

            if board.is_en_passant(move):
                piece_death_heatmaps[chess.PAWN][row][col] += 1
            else:
                victim = board.piece_at(dest)
                ptype = victim.piece_type
                if ptype != chess.KING:
                    piece_death_heatmaps[ptype][row][col] += 1
        
        board.push(move)

    if board.is_checkmate():
        loser = board.turn
        king_square = board.king(loser)
        
        if king_square is not None:
            row, col = 7 - chess.square_rank(king_square), chess.square_file(king_square)
            piece_death_heatmaps[chess.KING][row][col] += 1

In [50]:
def game_analyze(game: chess.pgn.Game):
    """analyze single game"""
    count_death(game)

In [51]:
game_count, game_valid = 0, 0

with open(pgn_filename) as pgn_file:
    with tqdm(
        total=GAMES_TO_PROCESS, desc="Process", unit="game", colour="green"
    ) as pbar:
        while game_count < GAMES_TO_PROCESS:
            pbar.update(1)
            game_count += 1

            game = chess.pgn.read_game(pgn_file)
            game_meta = chess.pgn.read_headers(pgn_file)

            # get elo of player
            white_elo = -1 if "?" in (t := game_meta["WhiteElo"]) else int(t)
            black_elo = -1 if "?" in (t := game_meta["BlackElo"]) else int(t)

            if white_elo < 0 or black_elo < 0:
                break

            game_analyze(game)

            game_valid += 1

Process: 100%|██████████| 500/500 [00:01<00:00, 487.88game/s]


In [52]:
piece_death_heatmaps[chess.PAWN]

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [ 41.,  46.,  28.,   9.,  10.,  65.,  52.,  38.],
       [ 37.,  35.,  85.,  71.,  82.,  76.,  65.,  46.],
       [ 41.,  98., 126., 400., 253., 109.,  84.,  46.],
       [ 33.,  91., 135., 329., 212.,  92.,  91.,  35.],
       [ 30.,  43.,  76.,  48.,  43.,  47.,  43.,  47.],
       [ 45.,  56.,  43.,   8.,   3.,  48.,  32.,  27.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]])

In [53]:
import matplotlib.pyplot as plt
import seaborn as sns

